In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, isnan
from pyspark.ml.feature import StringIndexer

# Step 1: Create Spark session
spark = SparkSession.builder \
    .appName("Data Cleaning") \
    .getOrCreate()

# Step 2: Load the dataset
file_path = "Desktop/big data visualization/dataset/catagaries_dataset.csv"
data = spark.read.csv(file_path, header=True, inferSchema=True)


# Step 4: Data Cleaning

# 4.1: Identify and remove duplicate rows
# Step 3: Identify duplicate rows
duplicates = data.groupBy(data.columns).count().filter("count > 1")
print("Duplicate Rows:")
duplicates.show()

# Step 4: Remove duplicate rows
data = data.dropDuplicates()
print("Duplicate successfully removed")

# Order by date 
data = data.orderBy("date_time")
print("Date time oder successfully")

# Step 3: Identify duplicate rows
duplicates = data.groupBy(data.columns).count().filter("count > 1")
print("Duplicate Rows:")
duplicates.show()

# 4.2: Identify and handle null or missing values

# Separate numeric and non-numeric columns
numeric_columns = [field.name for field in data.schema.fields if str(field.dataType) in ['DoubleType', 'IntegerType', 'FloatType']]
non_numeric_columns = [field.name for field in data.schema.fields if field.name not in numeric_columns]

# Check nulls for all columns, and isnan only for numeric columns
null_counts = data.select([
    count(when(col(c).isNull(), c)).alias(c) for c in non_numeric_columns
] + [
    count(when(col(c).isNull() | isnan(col(c)), c)).alias(c) for c in numeric_columns
])
print("Null Counts Per Column:")
null_counts.show()

# Step 5: Export the cleaned and transformed dataset to CSV
output_path = "Desktop/big data visualization/dataset/cleaned_transformed_dataset.csv"

try:
    data.toPandas().to_csv(output_path, index=False)
    print(f"Cleaned and transformed dataset saved successfully at {output_path}")
except Exception as e:
    print(f"Error saving dataset: {e}")

# Step 6: Stop the Spark session
spark.stop()


Duplicate Rows:
+-------+------------------+-------+-------+----------+------------+--------------------+-------------------+--------------+------------------+-------------+-------------------------+-----+
|holiday|              temp|rain_1h|snow_1h|clouds_all|weather_main| weather_description|          date_time|traffic_volume|weather_main_index|holiday_index|weather_description_index|count|
+-------+------------------+-------+-------+----------+------------+--------------------+-------------------+--------------+------------------+-------------+-------------------------+-----+
|   None|            294.52|    0.0|    0.0|         1|       Clear|        sky is clear|2017-06-30 11:00:00|          4725|               1.0|          0.0|                      0.0|    2|
|   None|            286.29|    0.0|    0.0|         1|       Clear|        sky is clear|2015-09-30 19:00:00|          3679|               1.0|          0.0|                      0.0|    2|
|   None|            287.86|    0.